*   目标
将基本的 Transformer 架构进行输入转换和位置编码定制，并加入上下文信息，从而使其能够有效地处理和预测复杂的时空（Vision）数据

* 实现Traffic Accident Risk Forecasting using Contextual Vision Transformers这篇论文提到的内容.我所用的数据集是预处理好的,有纽约和芝加哥两个城市,但我们先只用芝加哥这个城市.每个城市有12份文件.并且先只训练fine这个细粒度,并没有管另外六个文件(属于粗粒度)

*   输入数据-分成粗糙和细腻各6个文件
主输入是$T\times d \times I \times J$   时间×维度×二维网格位置
这是时间序列,其他都是静态信息


---
* **数据集**：芝加哥 (Chicago) 交通事故数据集。
* **实验范围**：本次实验先只做于 **Fine-grained (细粒度)** 尺度的预测。
* **输入数据**：
    * **动态数据**：历史事故风险图（$T \times D \times H \times W$）。
    * **静态数据**：区域的固有属性（如功能区分布）和路网拓扑图(用于将数据从grid和node间转换)。


In [29]:
#from google.colab import drive  #用于从drive中拿数据集
import os
import random
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score,average_precision_score
from tqdm.auto import tqdm

设立随机种子,确保结果可复现

In [30]:
def set_seed(seed=42):
    random.seed(seed)                  # 固定 Python 的随机数种子
    np.random.seed(seed)               # 固定 numpy 的随机数种子
    torch.manual_seed(seed)            # 固定 CPU 上 torch 的随机数
    torch.cuda.manual_seed_all(seed)   # 固定所有 GPU 上 torch 的随机数
    torch.backends.cudnn.deterministic = True  # 让 cudnn 算子确定性 (结果可复现, 但可能变慢)
    torch.backends.cudnn.benchmark = False     # 禁用自动选择最快算法; 若输入尺寸固定可设 True 加速

set_seed(42)

In [31]:
# _#准备工作
# print("--- 正在挂载 Google Drive ---")
# drive.mount('/content/drive',force_remount=True)   #挂载drive
# print("------------------------------")

# #拿文件
# CHI_path = '/content/drive/MyDrive/深度学习大作业/preprocessed_data/CHI'    #对应CHI外层文件夹
# NYC_path='/content/drive/MyDrive/深度学习大作业/preprocessed_data/NYC'  #对应NYC外层文件夹

CHI_path = 'preprocessed_data/CHI'    #对应CHI外层文件夹
NYC_path='preprocessed_data/NYC'  #对应NYC外层文件夹

print("CHI 路径下的内容：")
for item in os.listdir(CHI_path):
    print(item)

print("\nNYC 路径下的内容：")
# 列出 NYC_path 下的所有文件和文件夹
for item in os.listdir(NYC_path):
    print(item)

CHI 路径下的内容：
new_grid_data_c_4d.npy
new_grid_data_f_4d.npy
new_grid_node_map_c.npy
new_grid_node_map_f.npy
new_poi_adj_matrix_c.npy
new_poi_adj_matrix_f.npy
new_risk_adj_matrix_c.npy
new_risk_adj_matrix_f.npy
new_road_adj_matrix_c.npy
new_road_adj_matrix_f.npy
new_static_feat_c.npy
new_static_feat_f.npy

NYC 路径下的内容：
new_grid_data_c_4d.npy
new_grid_data_f_4d.npy
new_grid_node_map_c.npy
new_grid_node_map_f.npy
new_poi_adj_matrix_c.npy
new_poi_adj_matrix_f.npy
new_risk_adj_matrix_c.npy
new_risk_adj_matrix_f.npy
new_road_adj_matrix_c.npy
new_road_adj_matrix_f.npy
new_static_feat_c.npy
new_static_feat_f.npy


测试文件是否能正常读取

In [32]:
file_test = CHI_path+'/new_grid_data_c_4d.npy'
data = np.load(file_test)
print(data[0:2])
print(f"Data shape: {data.shape}")
print(f"Data type: {data.dtype}")
print(f"Min value: {np.min(data)}")
print(f"Max value: {np.max(data)}")
print(f"Mean value: {np.mean(data)}")
print(f"Standard deviation: {np.std(data)}")

[[[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   ...
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]]

  [[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   ...
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e

有粗细两个通道  12个文件是分成6:6

In [33]:
# 粗
Coarse_File_Names = [
    'new_grid_data_c_4d.npy',
    'new_static_feat_c.npy',
    'new_poi_adj_matrix_c.npy',
    'new_road_adj_matrix_c.npy',
    'new_risk_adj_matrix_c.npy',
    'new_grid_node_map_c.npy'
]

# 细

Fine_File_Names = [
    'new_grid_data_f_4d.npy',
    'new_static_feat_f.npy',
    'new_poi_adj_matrix_f.npy',
    'new_road_adj_matrix_f.npy',
    'new_risk_adj_matrix_f.npy',
    'new_grid_node_map_f.npy'
]

读取粗细全部文件

In [34]:
file_dict={}

for file_name in Coarse_File_Names:
    file_path=CHI_path+'/'+file_name
    file_dict[file_name]=np.load(file_path)

for file_name in Fine_File_Names:
    file_path=CHI_path+'/'+file_name
    file_dict[file_name]=np.load(file_path)

In [35]:
print(len(file_dict.keys()))

12


先拆分数据集  时间序列 所以按时间分,shuffle随机打乱分配会让模型看到未来

*   训练集:前70%
*   验证集:中15%
*   测试集:后15%

In [36]:
# --- 修改代码块 1: 数据归一化与切分 ---

train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# 1. 读取原始数据
raw_data = file_dict['new_grid_data_f_4d.npy']
print(f'原始数据最大值: {np.max(raw_data)}')

# 重要的一批,不然4w多损失玩不了了
# 2. 关键修改: Log 归一化 (Log1p = log(x+1))
# 不用方差,均值的原因:既保留了 0 值（无事故），又抑制了极端值的破坏力
# 这会将 [0, 11442] 的范围压缩到 [0, 9.3] 左右，极大利于 Transformer 训练
data_norm = np.log1p(raw_data)
print(f'归一化后数据最大值: {np.max(data_norm)}')

total_size = data_norm.shape[0]
train_size = int(total_size * train_ratio)
val_size = int(total_size * val_ratio)

print('划分连续时间序列 (使用归一化后的数据)')
# 训练集
train_data = data_norm[:train_size]

# 验证集
valid_data = data_norm[train_size:train_size+val_size]

# 测试集
test_data = data_norm[train_size+val_size:]

print('划分时间序列完成')
print(f'train_data shape: {train_data.shape}')
print(f'valid_data shape: {valid_data.shape}')
print(f'test_data shape: {test_data.shape}')

原始数据最大值: 11442.4619140625
归一化后数据最大值: 9.345173835754395
划分连续时间序列 (使用归一化后的数据)
划分时间序列完成
train_data shape: (6148, 7, 10, 10)
valid_data shape: (1317, 7, 10, 10)
test_data shape: (1319, 7, 10, 10)


将连续时间序列切分为 X (历史输入) 和 Y (未来标签) 样本.
---
但是这里要考虑过去时间步和预测未来时间步
* 看多少过去的数据,预测多久未来:可以理解成一个滑动窗口,前history_steps是X,后future_steps是Y
* 最终切分成的x和y矩阵是多对1
* 输入 [B, d ,H , W]
* 输出 [B , steps ,d ,H , W]

In [37]:
#规定时间步
history_steps = 6
future_steps = 1

def split_data_x_y(data,history_steps,future_steps):
    length = data.shape[0]
    X , Y = [] , []

    for i in range(length - history_steps - future_steps + 1): #滑动窗口
        X.append(data[i : i + history_steps])
        Y.append(data[i + history_steps : i + history_steps + future_steps])

    return np.array(X), np.array(Y)


print('对三个数据集分别进行切分')
X_train,Y_train = split_data_x_y(train_data,history_steps,future_steps)
X_valid,Y_valid = split_data_x_y(valid_data,history_steps,future_steps)
X_test,Y_test = split_data_x_y(test_data,history_steps,future_steps)


print('切分完成')
print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')
print(f'X_valid shape: {X_valid.shape}')
print(f'Y_valid shape: {Y_valid.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'Y_test shape: {Y_test.shape}')

对三个数据集分别进行切分
切分完成
X_train shape: (6142, 6, 7, 10, 10)
Y_train shape: (6142, 1, 7, 10, 10)
X_valid shape: (1311, 6, 7, 10, 10)
Y_valid shape: (1311, 1, 7, 10, 10)
X_test shape: (1313, 6, 7, 10, 10)
Y_test shape: (1313, 1, 7, 10, 10)


这里的时间特征和下面模型中定义的位置编码含义不同,这里是为了实现样本之间的时间顺序,后者是单个样本内的顺序

In [38]:
# --- 生成真实的时间特征 ---

def generate_global_time_features(time_features):  # 这里生成的是所有数据的时间特征(包含三个数据集)
    # 这里我们假设数据是连续的小时数据 (索引 0 = 第0小时)   看了部分原数据猜的
    indices = np.arange(time_features)   #[0,1,2,3,4,5,6,....................dataset_length-1]

    # 计算小时 (0-23) 和 星期 (0-6)
    hours = indices % 24      #
    days = (indices // 24) % 7

    # 使用 Sin/Cos 编码 (4个特征)
    # 这是transformer中经典的位置编码,因为transformer扔掉了rnn,没有时间性了
    time_feat = np.stack([     # stack就是说把一群东西塞成一个东西
        np.sin(2 * np.pi * hours / 24), # Hour_Sin
        np.cos(2 * np.pi * hours / 24), # Hour_Cos
        np.sin(2 * np.pi * days / 7),   # Week_Sin
        np.cos(2 * np.pi * days / 7)    # Week_Cos
    ], axis=1)

    return time_feat

# 生成全局时间表 (对应原始数据的 8784 行)      # 因为每个数据就是一个时间,所以说要生成样本数量的时间特征
global_time_features = generate_global_time_features(file_dict['new_grid_data_f_4d.npy'].shape[0])

# 根据索引提取时间作为每个数据集的时间特征
def each_dataset_time_features(start_index, data_length, history_steps):
    X_time = []
    # 循环次数(产生的时间特征数量)必须与 split_data_x_y 中的逻辑完全一致
    # 每个数据集样本数量是: length - history_steps - future_steps + 1
    epoches = data_length - history_steps - future_steps + 1

    for i in range(epoches):
        current_idx = start_index + i
        # 取出对应的 history_steps 长度的时间特征  因为每个样本要看前面的history_steps步长
        X_time.append(global_time_features[current_idx : current_idx + history_steps])

    return np.array(X_time)

print("正在重新生成时间特征...")

# 生成特征
Time_train = each_dataset_time_features(0, train_data.shape[0], history_steps)
Time_val = each_dataset_time_features(train_size, valid_data.shape[0], history_steps)
Time_test = each_dataset_time_features(train_size + val_size, test_data.shape[0], history_steps)

print(f"Time_train shape: {Time_train.shape}")

正在重新生成时间特征...
Time_train shape: (6142, 6, 4)


还有其他一些静态数据,这些数据特点:不是时间序列,也就是训练一个样本需要一整个静态数据

---

为了简化实验,先只调入一个静态数据用作模型训练


---

上面以前的写的,现在用了全部数据

In [39]:
static_data_raw = file_dict['new_static_feat_f.npy']
new_static_feat_f = torch.tensor(static_data_raw, dtype=torch.float32)

print(f"静态特征张量的形状: {new_static_feat_f.shape}")

静态特征张量的形状: torch.Size([10, 10, 4])


现在我们来导入其他文件


---

这四个文件不同于grid和static可作为Transformer的输入;它们是整个区域的地图只能通过模型内部 GCN 模块的连接矩阵

---

前三个是图,最后的grid_node能支持网格往返图  看它的格式是(100,63)


---
就是说我们本来是直接用static_feat的,现在我们用grid_node_map先转成图结构,然后与
1. poi_adj_matrix
2. road_adj_matrix
3. risk_adj_matrix
这三个文件融合,之后再通过grid_node_map转回网格结构,输入transformer


In [40]:
poi_adj_matrix = file_dict['new_poi_adj_matrix_f.npy']
road_adj_matrix = file_dict['new_road_adj_matrix_f.npy']
risk_adj_matrix = file_dict['new_risk_adj_matrix_f.npy']
grid_node_map = file_dict['new_grid_node_map_f.npy']

adj_tensors = {
    'poi': poi_adj_matrix,
    'road': road_adj_matrix,
    'risk': risk_adj_matrix,
    'grid_node_map': grid_node_map
}

print(f"POI邻接矩阵形状: {poi_adj_matrix.shape}")
print(f"道路邻接矩阵形状: {road_adj_matrix.shape}")
print(f"风险邻接矩阵形状: {risk_adj_matrix.shape}")
print(f"网格节点映射形状: {grid_node_map.shape}")

POI邻接矩阵形状: (63, 63)
道路邻接矩阵形状: (63, 63)
风险邻接矩阵形状: (63, 63)
网格节点映射形状: (100, 63)


用dataset继续创建数据集,不同于一般的x和y,这里还需要加上我们新创建的时间特征和静态特征

---

需要新写类,继承dataset,因为原dataset只支持两个参数

In [41]:
class FullDataset(Dataset):
    def __init__(self, X, Y, time_feature,static_data):
        self.X = torch.tensor(X,dtype=torch.float32)
        self.Y = torch.tensor(Y,dtype=torch.float32)
        self.time_feature = torch.tensor(time_feature,dtype=torch.float32)

        self.static_data = static_data

    def __len__(self):
        return len(self.X)

    def __getitem__(self,index):
        return self.X[index], self.Y[index], self.time_feature[index],self.static_data  #共享就是体现在这里,没有[index],每个样本都需要完整的static
        # 这样后在dataloader中拿static_data 格式就变成[B, 10 , 10, 4]

生成三个dataset

In [42]:
train_dataset = FullDataset(X_train, Y_train, Time_train,new_static_feat_f)
val_dataset = FullDataset(X_valid, Y_valid, Time_val,new_static_feat_f)
test_dataset = FullDataset(X_test, Y_test, Time_test,new_static_feat_f)

 开始dataloader

In [43]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  #这里不会影响数据的时序性,滑动窗口切分（将连续的 history_steps 步作为 X）已经将每个样本（X, Y, Time）变成了独立的预测事件,所以打乱没影响
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)   #到时候写循环时,可以用for循环,一下子拿到多个数据

print(len(train_loader))
print(len(val_loader))
print(len(test_loader))

192
41
42


模型结构定义

---


1.   将输入时空风险图 [B, history_steps, D, H, W] 转换成 Transformer 的 Tokens [B, Sequence_length, D_model] ----输入预处理

> 用一个 2D 卷积操作同时完成了空间分块 (Patching) 和 特征投影 (Embedding)



> 空间区域的数量视为序列的长度(Sequence_length) 这是最后的H/P和W/P



---


2.   还需要完成图卷积层(GCN) 实现将static信息转成图结构,用gcn将其与三个文件的结构信息融合,生成 增强后的静态特征 ($X_{\text{static\_GCN}}$)


---


3.   ST-ViT 模型，用于时空风险预测


> 引入了空间位置编码和时间位置编码,实际一张图我们分成了(Grid_size//Kernel_size)**2个小格子,每个小格子又有6个时间 所以一个序列的长度是互乘




In [44]:
# # 作为模型的子模块,用来处理 X_train (风险特征图)  使其符合transformer的输入格式
# class PatchEmbedding(nn.Module):
#     def __init__(self, d_model, D, history_steps, kernel_size):
#         #d_model:transformer内部维度,也是我们要转换成的
#         #D:风险特征的维度,也就是(6148, 7, 10, 10)中的7
#         #history_steps:历史步信息,回过头看的信息
#         #patch_size:分块大小

#         super().__init__()

#         # Conv2d 的输入通道数是 D * history_steps

#         # 重要:  这里因为考虑到卷积是只能看到Channels×H×W的图像,所以我们让风险特征维度和历史步推叠起来
#         # 使得单次卷积能够看到所有历史和所有特征维度
#         in_channels_conv = history_steps*D

#         self.conv = nn.Conv2d(
#             in_channels=in_channels_conv,
#             out_channels=d_model,  # 输出通道数就是 D_model,转成transformer内部维度
#             kernel_size=kernel_size,
#             stride=kernel_size  #kernel_size和stride相同可以保证卷积不会重复
#         )

#     def forward(self, x):   #维度参考之前的处理 X_train shape: (6142, 6, 7, 10, 10)
#         # 1. 调整维度(堆叠)：[B, history_steps, D, H, W] -> [B, history_steps*D, H, W]
#         # transpose可有可无,习惯上让D优先于history_steps
#         x = x.transpose(1, 2).flatten(1, 2)

#         # 2. 卷积操作：[B, history_steps*D, H, W] -> [B, D_model, H/P, W/P]  这里实现把一张图进行分块
#         x = self.conv(x)

#         # 3. 展平和转置：[B, D_model, H/P, W/P] -> [B, Num_Patches, D_model]  得到序列长度,每一个小块是一个词,合起来就是一个序列
#         # 这一步完成了 Token 的构建
#         # x.flatten(2) 会展平从索引 2 开始到张量末尾的所有维度 让H/P和W/P相乘
#         x = x.flatten(2).transpose(1, 2)
#         return x

# PatchEmbedding 应该独立处理每一个时间步，保留时间维度，让 Transformer 去做时序融合

---

上面的丢弃

---

旧逻辑： 把 $6$ 个时间步堆叠成通道，一次卷积全看完 $\rightarrow$ 时间维度消失。新逻辑： 把 $6$ 个时间步视为 $6$ 张独立的图片，分别卷积 $\rightarrow$ 保留时间维度 [B, T, N, D]。

In [45]:
# 在这里处理主输入
class PatchEmbedding(nn.Module):
    def __init__(self, d_model, D, kernel_size): # 注意：这里去掉了 history_steps 参数
        super().__init__()
        # d_model: transformer内部维度
        # D: 风险特征的维度 (7)
        # kernel_size: 分块大小 (1)

        # 输入通道数改为 D (7)，不再是 history_steps * D
        # 我们要对每个时间步独立做卷积
        self.conv = nn.Conv2d(
            in_channels=D,
            out_channels=d_model,
            kernel_size=kernel_size,
            stride=kernel_size
        )

    def forward(self, x):
        # x shape: [B, history_steps, D, H, W]
        B, T, D, H, W = x.shape

        # 1. 合并 Batch 和 Time 维度，把每帧当做独立图片处理
        # 原来是把TD融合,最后能通过view恢复时间维度,但是原来那样会把时间维度干消失掉
        # [B, T, D, H, W] -> [B*T, D, H, W]
        x = x.view(B * T, D, H, W)

        # 2. 卷积操作 (Patching + Embedding)
        # [B*T, D, H, W] -> [B*T, d_model, H/P, W/P]
        x = self.conv(x)

        # 3. 展平空间维度并转置，生成 Tokens
        # [B*T, d_model, H/P, W/P] -> [B*T, d_model, Num_Patches] -> [B*T, Num_Patches, d_model]
        x = x.flatten(2).transpose(1, 2)

        # 4. 恢复 Batch 和 Time 维度
        # [B*T, Num_Patches, d_model] -> [B, T, Num_Patches, d_model]
        x = x.view(B, T, -1, x.shape[-1])

        return x

下面解决四个静态信息融合(除去grid主输入和grid_node_map)

In [46]:
# 定义图卷积层
# 对图上的每个节点，聚合它所有邻居的特征信息，并将这些信息融入到该节点自己的特征中
# 先把static的特征转成D_model,再与adj相乘  让四个信息融合
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True):  #传进去的参数为了把static的特征维度变成D_model
        super().__init__()
        self.linear = nn.Linear(in_features, out_features, bias=bias)

    def forward(self, x, adj):     # 把batch中每一个样本去和adj矩阵相乘
        # x: [节点数, 特征数] [63,4]
        # adj: [节点数, 节点数] [63,63]

        # 1. 把static从当前维度转成D_model  new_static=[63,64]
        new_static = self.linear(x)

        # 2. 矩阵乘法 (图卷积，即邻居聚合)  不用相加 这是图信号处理中 唯一的 正确方式，它将邻接矩阵 A 作为 信息聚合的滤波器，实现了对邻居特征的加权求和
        # 相加就可以理解成都糅合在一起了,信息乱掉了  ; 注意区分adj3个文件是可以相加的,它们是固定信息
        output = torch.matmul(adj, new_static)  #[63,64]
        return output

# 借助上面的GCN正式处理statci和adj
# --- 修改代码块 3: GCN 增加归一化 ---

class StaticContextGCN(nn.Module):
    def __init__(self, grid_size, map_tensor, adj_tensors, static_input_dim, D_model):
        super().__init__()

        self.grid_size = grid_size
        self.grid_node_map = map_tensor

        self.gcn_layer = GCNLayer(static_input_dim, D_model)
        self.relu = nn.ReLU()

        # 1. 初始邻接矩阵相加  就是把4个文件中除了grid_node_map相加
        raw_adj = (adj_tensors['poi'] + adj_tensors['road'] + adj_tensors['risk'])

        # 2. 关键修改: 计算归一化的拉普拉斯矩阵 (D^-0.5 * A * D^-0.5)
        # 这里能保证数值稳定,跟上面dataset取log差不多

        self.norm_adj = self.calculate_laplacian(raw_adj)

    def calculate_laplacian(self, adj):   # 这边目的是实现拉普拉斯
        # 添加自环 (Self-loop)，防止除零并保留自身特征
        adj = adj + torch.eye(adj.size(0), device=adj.device)
        # 计算度矩阵 D
        row_sum = adj.sum(1)
        d_inv_sqrt = torch.pow(row_sum, -0.5)
        d_inv_sqrt[torch.isinf(d_inv_sqrt)] = 0.
        d_mat_inv_sqrt = torch.diag(d_inv_sqrt)

        # 对称归一化
        return torch.matmul(torch.matmul(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)


    def forward(self, x_static):
        # x_static: [B, 10, 10, 4]
        batch_size = x_static.size(0)

        # 由于静态特征所有样本都一样，我们只算一次图卷积，然后广播
        # 取第一个样本: [10, 10, 4] -> [100, 4]
        flat_static = x_static[0].flatten(0, 1)

        # 映射到图节点: [100, 4]转置 * [100, 63] -> [4, 63] ->转置-> [63, 4]  注意用transpose调整顺序,特征数量往后放
        node_features = torch.matmul(flat_static.transpose(0, 1), self.grid_node_map).transpose(0, 1)

        # GCN 运算
        # [63, 4] * [63, 63] -> ... -> [63, D_model]  还是在图节点中
        gcn_out = self.relu(self.gcn_layer(node_features, self.norm_adj))

        # 映射回网格  目的就是63->100
        # [63, D_model]转置 * [63, 100] -> [D_model, 100] -> [100, D_model]
        grid_features = torch.matmul(gcn_out.transpose(0, 1), self.grid_node_map.transpose(0, 1)).transpose(0, 1)

        # 扩展回 Batch 维度: [B, 100, D_model]  unsqueeze实现0号位有维度,repeat实现广播至批次数量
        return grid_features.unsqueeze(0).repeat(batch_size, 1, 1)

在STViT_FineGrained中实现上面所述功能的组装

In [47]:
class STViT_FineGrained(nn.Module):
    def __init__(self, d_model, n_heads, n_layers, D, history_steps, future_steps,
                 grid_size, kernel_size, num_patches, time_feature_dim,
                 static_feature_dim, adj_tensors):
        super().__init__()

        # 1. 修改：PatchEmbedding 不再需要 history_steps 参数
        self.patch_embed = PatchEmbedding(d_model, D, kernel_size)

        self.time_embed = nn.Linear(time_feature_dim, d_model)      # 时间特征 (4 -> 64)

        # GCN 模块 (保持不变)
        self.static_context_gcn = StaticContextGCN(
            grid_size=grid_size,
            map_tensor=adj_tensors['grid_node_map'],
            adj_tensors={k: v for k, v in adj_tensors.items() if k != 'grid_node_map'},
            static_input_dim=static_feature_dim,
            D_model=d_model
        )

        # 参数保存一下
        self.num_patches = num_patches
        self.D = D
        self.history_steps = history_steps
        self.future_steps = future_steps
        self.grid_size = grid_size

        # 位置编码  transformer必须的,不同于上面的时间特征,这里是'单句话中各个词之间的顺序'
        # 开头设置成1是用于广播
        self.spatial_pos_embedding = nn.Parameter(torch.randn(1, num_patches, d_model))
        self.temporal_pos_embedding = nn.Parameter(torch.randn(1, history_steps, d_model))

        # Transformer每个层
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * 4,  # 一般来讲,transormer内部前馈网络的维度就是D_model*4
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        # 预测头
        flat_input_dim = history_steps * num_patches * d_model
        flat_output_dim = future_steps * D * grid_size**2
        self.prediction_head = nn.Sequential(
            nn.Linear(flat_input_dim, 256),
            nn.GELU(),
            nn.Linear(256, flat_output_dim)
        )

    def forward(self, x_grid, x_time, x_static):
        batch_size = x_grid.size(0)

        # 1. Patch Embedding  主要输入
        # 输入: [B, T=6, D, H, W]
        # 输出: [B, T=6, N=100, D_model=64] (已经包含时间维度)
        patch_embed = self.patch_embed(x_grid)

        # 2. 特征嵌入(时间特征)  统一度量衡
        # 时间嵌入: x_time 是 [B, 6, 4]
        # 输出: [B, 6, 64]
        time_embed = self.time_embed(x_time)

        # 静态 GCN: x_static 是 [B, 10, 10, 4]
        # 输出: [B, 100, 64] (静态特征，没有时间维度)
        gcn_static_tokens = self.static_context_gcn(x_static)

        # 3. 序列构造与融合

        # a. 主序列：[B, 6, 100, 64] -> [B, 600, 64]   100个格子,每个格子6个时间  所以总'词'数为600
        # 我们直接展平 Time 和 Spatial 维度
        sequence = patch_embed.flatten(1, 2)

        # b. 上下文融合
        # 格式都要搞成sequence[B, 600, 64]的样式以便相加
        # 时间上下文: [B, 6, 64] -> [B, 6, 1, 64] -> 复制100次 -> [B, 6, 100, 64] -> [B, 600, 64]
        # 将每个时间步的特征广播到该时间步的所有 Patch 上  切忌6直接repeat->600
        time_context = time_embed.unsqueeze(2).repeat(1, 1, self.num_patches, 1).flatten(1, 2)

        # 静态上下文: [B, 100, 64] -> [B, 1, 100, 64] -> 复制6次 -> [B, 6, 100, 64] -> [B, 600, 64]
        # 将静态特征广播到所有时间步上
        gcn_context = gcn_static_tokens.unsqueeze(1).repeat(1, self.history_steps, 1, 1).flatten(1, 2)

        # 融合所有特征
        sequence = sequence + time_context + gcn_context

        # c. 位置编码融合
        # 时间 PE: [1, 6, 64] -> repeat_interleave -> [1, 600, 64] (1,1,..,1, 2,2,..,2)
        # 注意时间要用repeat_interleave,因为不管是temp_pe还是spatial_pe都是以时间为主导的
        temp_pe = self.temporal_pos_embedding.repeat_interleave(self.num_patches, dim=1)
        # 空间 PE: [1, 100, 64] -> repeat -> [1, 600, 64] (1,2,..,100, 1,2,..,100)
        spatial_pe = self.spatial_pos_embedding.repeat(1, self.history_steps, 1)

        sequence = sequence + temp_pe + spatial_pe     # 所有数据处理完成

        # --- 4. Transformer ---
        transformer_output = self.transformer_encoder(sequence)    # [B,Sequence_length,D_model]

        # --- 5. Prediction Head ---
        flat_output = transformer_output.flatten(1)  # 展平后塞进全连接
        pred = self.prediction_head(flat_output)

        # 变成与Y相同的格式
        pred = pred.view(batch_size, self.future_steps, self.D, self.grid_size, self.grid_size)
        return pred

定义模型运行在什么设备上

In [48]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'当前模型运行在:{device}')

当前模型运行在:cuda


把剩余的4个文件转移到gpu中
提前传,不用等到训练中.这四个数据全局都在用

In [49]:
# 用.float() 类型     这四个数据进去后全在乘
poi_adj_matrix_tensor = torch.from_numpy(file_dict['new_poi_adj_matrix_f.npy']).float().to(device)
road_adj_matrix_tensor = torch.from_numpy(file_dict['new_road_adj_matrix_f.npy']).float().to(device)
risk_adj_matrix_tensor = torch.from_numpy(file_dict['new_risk_adj_matrix_f.npy']).float().to(device)

# 2. 网格节点映射 一般用Long/Int
# 修正：将其转换为 float 类型; 进去全在乘
grid_node_map_tensor = torch.from_numpy(file_dict['new_grid_node_map_f.npy']).float().to(device) # 将 .long() 改为 .float()

# 更新 adj_tensors 要让模型用tensor版本
adj_tensors['poi'] = poi_adj_matrix_tensor
adj_tensors['road'] = road_adj_matrix_tensor
adj_tensors['risk'] = risk_adj_matrix_tensor
adj_tensors['grid_node_map'] = grid_node_map_tensor

定义超参数

In [50]:
D_model=64   #transformer内部的工作维度
N_heads=8    #多头注意力
N_layers=3   #Encoder层数  编码器重复几次
# History_steps=6  #历史步长  这两部分在之前生成X_train...的时候就有了
# Future_steps=1      #预测步长
D=7          #风险维度
Grid_size=10  #网格大小,长宽都是10
Kernel_size = 1   # 补丁大小-卷积大小和步长大小
Num_patchs=(Grid_size//Kernel_size)**2  #每张图有几个块
Time_feature_dim=4  #时间特征维度
Static_feature_dim=4  #静态特征维度

实例化模型

In [51]:
model=STViT_FineGrained(D_model,N_heads,N_layers,D,history_steps,future_steps,Grid_size,Kernel_size,Num_patchs,Time_feature_dim,Static_feature_dim,adj_tensors)
model.to(device)

STViT_FineGrained(
  (patch_embed): PatchEmbedding(
    (conv): Conv2d(7, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (time_embed): Linear(in_features=4, out_features=64, bias=True)
  (static_context_gcn): StaticContextGCN(
    (gcn_layer): GCNLayer(
      (linear): Linear(in_features=4, out_features=64, bias=True)
    )
    (relu): ReLU()
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (d

###  损失函数与优化器

**零膨胀**
在交通数据中，90% 以上的网格在大部分时间都是**没有事故 (0)** 的。
如果我们使用普通的 MSE 损失，模型会倾向于“偷懒”——全部预测为 0 就能得到很小的误差。

**解决方案：加权 MSE (Weighted MSE)**
我们自定义了损失函数，给**真实发生事故 (Target > 0)** 的样本赋予 **20倍** 的权重。
* 这迫使模型必须关注那些稀疏但重要的“高风险区域”，而不是单纯追求平均误差最小。


---

这真的很重要,不然根本预测不出来,acc全是0,🤮

In [52]:
# --- 修改位置：自定义加权损失函数 ---

class WeightedMSELoss(nn.Module):
    def __init__(self, non_zero_weight=10.0):
        super().__init__()
        self.non_zero_weight = non_zero_weight

    def forward(self, pred, target):
        # 计算标准 MSE: (pred - target)^2
        loss = (pred - target) ** 2

        # 权重矩阵
        # 如果 target > 0 (有事故)，权重设为 non_zero_weight (例如 10)
        # 如果 target == 0 (无事故)，权重设为 1
        weights = torch.ones_like(target)   # 创建好权重矩阵(全1),默认都是无事故
        weights[target > 0] = self.non_zero_weight  #给事故点赋值

        # 应用权重
        weighted_loss = loss * weights

        # 返回平均损失
        return weighted_loss.mean()

# 使模型关注稀疏的事故数据
criterion = WeightedMSELoss(non_zero_weight=20.0) # 权重可以尝试 10 或 20

# 优化器
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# 学习率衰减策略 (Scheduler)  无所谓
# 损失在patience个epoch内不再下降,就将lr×factor衰减
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

训练和验证主循环

In [53]:
def train_epoch(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0

    tqdm_ob = tqdm(dataloader, desc=f"Training", leave=False)

    for x_grid, y_target, x_time, x_static in tqdm_ob:
        x_grid = x_grid.to(device)
        y_target = y_target.to(device)
        x_time = x_time.to(device)
        x_static = x_static.to(device)

        optimizer.zero_grad()

        # 前向传播
        output = model(x_grid, x_time, x_static)

        # 计算损失
        loss = criterion(output, y_target)

        # 反向传播
        loss.backward()

        # --- 关键修复：梯度裁剪 ---      确实很重要,不然练的时候💣了
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # 优化
        optimizer.step()

        # 累加损失
        total_loss += loss.item() * x_grid.size(0)

        tqdm_ob.set_postfix({'Batch Loss': f'{loss.item():.4f}'})

    return total_loss / len(dataloader.dataset)  # 返回的是整个的平均损失

验证集的指标用rmse和recall

---

1.  **回归指标 (RMSE)**：均方根误差。用于衡量预测数值的准确度（越低越好）。
2.  **分类指标 (Recall@TopK)**：
    * **逻辑**：我们不看具体的数值准不准，而是看模型能否把**真正会发生事故的区域**排在预测列表的前面。
    * **Top 5% Recall**：如果我们向预测风险最高的 5% 的区域派遣警力，能覆盖多少真实的事故？（越高越好）。

In [54]:
# --- 修改代码块 4: 验证时反归一化 ---

def validate_epoch(model, dataloader, criterion, top_k=50):
    model.eval()
    total_weighted_loss = 0

    # 存储所有批次的预测值和真实值，用于计算全局指标
    all_preds_real = []
    all_targets_real = []

    tqdm_ob = tqdm(dataloader, desc=f"Validating", leave=False)

    with torch.no_grad():
        for x_grid, y_target, x_time, x_static in tqdm_ob:
            x_grid, y_target, x_time, x_static = x_grid.to(device), y_target.to(device), x_time.to(device), x_static.to(device)

            output = model(x_grid, x_time, x_static)    # shape类似Y:(6142, 1, 7, 10, 10)

            # 1. 计算加权 Loss
            loss = criterion(output, y_target)
            total_weighted_loss += loss.item() * x_grid.size(0)

            # 2. 反归一化
            real_pred = torch.expm1(output).detach()  # detach从计算图中“分离”张量，返回一个不需要梯度、不参与反向传播的新张量
            real_target = torch.expm1(y_target).detach()
            real_pred = torch.clamp(real_pred, min=0)       # clamp将张量中的值限制在指定范围内（这里是下限为 0，无上限）

            all_preds_real.append(real_pred)
            all_targets_real.append(real_target)

    # 合并所有 batch 数据 [Total_Samples, T, D, H, W]    拼接起来成一起
    all_preds_real = torch.cat(all_preds_real, dim=0)
    all_targets_real = torch.cat(all_targets_real, dim=0)

    # --- 指标计算 1: RMSE (与论文对比) ---
    mse = torch.mean((all_preds_real - all_targets_real) ** 2)
    rmse = torch.sqrt(mse).item()

    # --- 指标计算 2: Recall@K (论文标准) ---
    # 将所有时空格子的预测值展平 [Total_Samples * T * D * H * W]      转成1维的好运算
    flat_preds = all_preds_real.view(-1)
    flat_targets = all_targets_real.view(-1)

    # 求出总事故数
    total_accidents = (flat_targets > 0).sum().item()

    if total_accidents == 0:
        recall_k = 0.0
    else:
        # 取预测值最高的 Top-K 个格子的索引
        # 注意：这里 K 通常取总格子数的一定比例，或者固定数值(如前500个高风险点)
        # 为了演示，我们取前 5% 的高风险格子作为 "预测有事故"
        k_count = int(flat_preds.numel() * 0.05)   # 知道5%有多少格子
        _, top_k_indices = torch.topk(flat_preds, k_count)   # 根据模型预测的风险值（flat_preds）从大到小排序，取出前 500 名的索引（位置 ID）。这些就是我们要派警察去的地方

        # 看看这 Top-K 个预测中，有多少是真的发生了事故
        hits = (flat_targets[top_k_indices] > 0).sum().item()

        # Recall@K = 命中的事故数 / 总事故数
        # 昨晚全城发生的事故中，有多少是被我们提前预测到并覆盖住的
        recall_k = hits / total_accidents

    # --- 指标计算 3: MAP (Mean Average Precision) ---
    y_true = (flat_targets > 0).cpu().numpy().astype(int)
    y_scores = flat_preds.cpu().numpy()

    # 计算 MAP
    # average_precision_score 本质上就是计算 AP
    # 这里的 y_scores 不需要自己手动排序，sklearn 会自动排
    map_score = average_precision_score(y_true, y_scores)

    print(f"MAP (平均精度均值): {map_score:.4f} (论文目标: ~0.0980)")

    avg_weighted_loss = total_weighted_loss / len(dataloader.dataset)

    print(f"\n[论文同款指标]")
    print(f"RMSE (均方根误差): {rmse:.4f} (论文目标: ~7.0353)")
    print(f"Recall@Top5% : {recall_k*100:.2f}% (论文目标: ~21.95%)")

    return avg_weighted_loss, rmse, recall_k , map_score

训练训练集,看看在验证集上如何

In [ ]:
epoches = 30

for epoch in range(epoches):
    print('---------------------------------------------')
    print(f'这是第 {epoch+1}/{epoches} 轮')

    # --- 训练 ---
    train_loss = train_epoch(model, train_loader, criterion, optimizer)

    # --- 验证 (设定阈值为 0.05) ---
    # 注意：这里接收了 3 个返回值
    val_loss, val_rmse, val_recall_k , val_map_score= validate_epoch(model, val_loader, criterion)

    # --- 更新学习率 ---
    # 基于 Validation Loss 调整学习率
    scheduler.step(val_loss)

    # 手动打印当前学习率
    current_lr = optimizer.param_groups[0]['lr']
    print(f'Current Learning Rate: {current_lr}')

    # --- 打印结果 ---
    print(f'Train Weighted Loss: {train_loss:.4f}')
    print(f'Val Weighted Loss:   {val_loss:.4f}')
    print(f'Val RMSE:            {val_rmse:.4f}') # 关注这个
    print(f'Val Recall@K:        {val_recall_k:.4f}') # 关注这个
    print(f'Val MAP:             {val_map_score:.4f}') # 关注这个

    print('---------------------------------------------')

---------------------------------------------
这是第 1/30 轮


Training:   0%|          | 0/192 [00:00<?, ?it/s]

Validating:   0%|          | 0/41 [00:00<?, ?it/s]

MAP (平均精度均值): 0.7337 (论文目标: ~0.0980)

[论文同款指标]
RMSE (均方根误差): 49.6098 (论文目标: ~7.0353)
Recall@Top5% : 28.75% (论文目标: ~21.95%)
Current Learning Rate: 0.001
Train Weighted Loss: 8.5494
Val Weighted Loss:   0.9749
Val RMSE:            49.6098
Val Recall@K:        0.2875
Val MAP:             0.7337
---------------------------------------------
---------------------------------------------
这是第 2/30 轮


Training:   0%|          | 0/192 [00:00<?, ?it/s]

Validating:   0%|          | 0/41 [00:00<?, ?it/s]

MAP (平均精度均值): 0.8059 (论文目标: ~0.0980)

[论文同款指标]
RMSE (均方根误差): 36.3457 (论文目标: ~7.0353)
Recall@Top5% : 31.79% (论文目标: ~21.95%)
Current Learning Rate: 0.001
Train Weighted Loss: 1.3794
Val Weighted Loss:   0.7117
Val RMSE:            36.3457
Val Recall@K:        0.3179
Val MAP:             0.8059
---------------------------------------------
---------------------------------------------
这是第 3/30 轮


Training:   0%|          | 0/192 [00:00<?, ?it/s]

可恶,这里recall竟然比论文还高,怎么可能呢?
看到rmse偏高,所以可以认为模型在大惊小怪 倾向于在所有可能发生事故的危险区域（比如市中心路口）都预测极高的风险值